In [1]:
import torch
from torch import nn

In [2]:
lrn = nn.LocalResponseNorm(size=3, alpha=1, beta=1, k=1)

In [ ]:

signal = torch.zeros(1, 5, 2, 2)

# Case A: High neighboring activity
signal[0, 1, :, :] = 20.0  # Neighbor 1
signal[0, 2, :, :] = 10.0  # THE TARGET CHANNEL
signal[0, 3, :, :] = 20.0  # Neighbor 2

output_suppressed = lrn(signal)

print(f"Channel 2 with LOUD neighbors before LRN: \n{signal}")
print(f"Channel 2 with LOUD neighbors after LRN: \n{output_suppressed}")

Channel 2 with LOUD neighbors before LRN: 
tensor([[[[ 0.,  0.],
          [ 0.,  0.]],

         [[20., 20.],
          [20., 20.]],

         [[10., 10.],
          [10., 10.]],

         [[20., 20.],
          [20., 20.]],

         [[ 0.,  0.],
          [ 0.,  0.]]]])
Channel 2 with LOUD neighbors after LRN: 
tensor([[[[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.1193, 0.1193],
          [0.1193, 0.1193]],

         [[0.0332, 0.0332],
          [0.0332, 0.0332]],

         [[0.1193, 0.1193],
          [0.1193, 0.1193]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]]]])


In [4]:
print(f"Before LRN, signal in channel 2 is bigger than in channel 3 by {signal[0, 1, 0, 0] / signal[0, 2, 0, 0]}")
print(f"After LRN, signal in channel 2 is bigger than in channel 3 by {output_suppressed[0, 1, 0, 0] / output_suppressed[0, 2, 0, 0]}")

Before LRN, signal in channel 2 is bigger than in channel 3 by 2.0
After LRN, signal in channel 2 is bigger than in channel 3 by 3.5904572010040283


In [ ]:
# Case B: Low neighboring activity 
signal_quiet = torch.zeros(1, 5, 2, 2)
signal_quiet[0, 1, :, :] = 0.1   # Quiet neighbor
signal_quiet[0, 2, :, :] = 10.0  # THE TARGET CHANNEL
signal_quiet[0, 3, :, :] = 0.1   # Quiet neighbor

output_strong = lrn(signal_quiet)


print(f"Channel 2 with QUIET neighbors before LRN: \n {signal_quiet}")
print(f"Channel 2 with QUIET neighbors after LRN: \n {output_strong}")

Channel 2 with QUIET neighbors before LRN: 
 tensor([[[[ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[ 0.1000,  0.1000],
          [ 0.1000,  0.1000]],

         [[10.0000, 10.0000],
          [10.0000, 10.0000]],

         [[ 0.1000,  0.1000],
          [ 0.1000,  0.1000]],

         [[ 0.0000,  0.0000],
          [ 0.0000,  0.0000]]]])
Channel 2 with QUIET neighbors after LRN: 
 tensor([[[[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.0029, 0.0029],
          [0.0029, 0.0029]],

         [[0.2912, 0.2912],
          [0.2912, 0.2912]],

         [[0.0029, 0.0029],
          [0.0029, 0.0029]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]]]])


In [6]:
print(f"Before LRN, signal in channel 2 is bigger than in channel 3 by {signal_quiet[0, 2, 0, 0] / signal_quiet[0, 1, 0, 0]}")
print(f"After LRN, signal in channel 2 is bigger than in channel 3 by {output_strong[0, 2, 0, 0] / output_strong[0, 1, 0, 0]}")

Before LRN, signal in channel 2 is bigger than in channel 3 by 100.0
After LRN, signal in channel 2 is bigger than in channel 3 by 99.99029541015625


By squashing the activations, LRN prevents the gradients from becoming too large (exploding) or too small (vanishing) in the early layers.